In [1]:
import pandas as pd

In [2]:
df_ZTF = pd.read_pickle('consolidated_dataset.pkl')

In [3]:
df_ZTF.head()

,oid,candid,ra,dec,science,reference,diff,class
11,ZTF21aaebkmj,1501128131215015004,65.163495,-26.636435,"[[404.2104, 394.65637, 403.2213, 391.0437, 396...","[[387.88876, 388.28632, 390.10416, 391.13345, ...","[[15.940292, 8.174846, 11.953584, 0.73741037, ...",sn
14,ZTF18abtnlik,643114394415015053,279.321654,49.148389,"[[79.67049, 86.467064, 78.972015, 81.246994, 8...","[[86.60097, 89.05083, 88.408226, 86.859764, 86...","[[-3.8708775, -6.8700647, -7.4733896, -4.98984...",sn
9,ZTF19acjurot,1058126442015015014,320.853375,-17.720127,"[[113.59538, 131.67825, 126.9975, 119.60596, 1...","[[121.8025, 123.31801, 121.95386, 122.05768, 1...","[[-9.240418, 11.035837, 4.2956905, -2.7166245,...",sn
15,ZTF20aanvpzo,1200441200215015003,224.375994,8.389928,"[[161.64427, 161.45685, 166.97305, 155.20792, ...","[[164.45232, 162.76022, 164.05763, 166.44301, ...","[[-2.7969754, -1.3325354, 0.4744801, -8.980967...",sn
19,ZTF20aawxpru,1306411870015015009,322.840280,9.429956,"[[147.37575, 151.22055, 142.37598, 139.72981, ...","[[148.13307, 144.70827, 142.08238, 140.17836, ...","[[-2.9152439, 6.9139023, 0.40033725, 0.6607923...",sn


In [4]:
df_ZTF_2 = df_ZTF.drop_duplicates(subset='oid').set_index('oid', drop=True)

In [6]:
df_ZTF_2.to_pickle("dataset_ZTF.pkl")


In [9]:
df_ZTF_2.head()

,candid,ra,dec,science,reference,diff,class
oid,,,,,,,
ZTF21aaebkmj,1501128131215015004,65.163495,-26.636435,"[[404.2104, 394.65637, 403.2213, 391.0437, 396...","[[387.88876, 388.28632, 390.10416, 391.13345, ...","[[15.940292, 8.174846, 11.953584, 0.73741037, ...",sn
ZTF18abtnlik,643114394415015053,279.321654,49.148389,"[[79.67049, 86.467064, 78.972015, 81.246994, 8...","[[86.60097, 89.05083, 88.408226, 86.859764, 86...","[[-3.8708775, -6.8700647, -7.4733896, -4.98984...",sn
ZTF19acjurot,1058126442015015014,320.853375,-17.720127,"[[113.59538, 131.67825, 126.9975, 119.60596, 1...","[[121.8025, 123.31801, 121.95386, 122.05768, 1...","[[-9.240418, 11.035837, 4.2956905, -2.7166245,...",sn
ZTF20aanvpzo,1200441200215015003,224.375994,8.389928,"[[161.64427, 161.45685, 166.97305, 155.20792, ...","[[164.45232, 162.76022, 164.05763, 166.44301, ...","[[-2.7969754, -1.3325354, 0.4744801, -8.980967...",sn
ZTF20aawxpru,1306411870015015009,322.840280,9.429956,"[[147.37575, 151.22055, 142.37598, 139.72981, ...","[[148.13307, 144.70827, 142.08238, 140.17836, ...","[[-2.9152439, 6.9139023, 0.40033725, 0.6607923...",sn


In [6]:
import pickle
import pandas as pd
from pathlib import Path
from astropy.io import fits

# Rutas
multires_path = 'multiresolution_fits_files'
output_pickle = 'multires_images.pkl'
ZTF_path = 'dataset_ZTF.pkl'

print("Cargando el dataset de ZTF...")
# Cargar el dataset de ZTF
with open(ZTF_path, "rb") as f:
    data_ZTF = pickle.load(f)

# Asegurar que sea un DataFrame
if not isinstance(data_ZTF, pd.DataFrame):
    data_ZTF = pd.DataFrame(data_ZTF)

# Lista de OIDs a procesar
oid_subset = data_ZTF.index.astype(str)

# Cargar progreso previo si existe
try:
    with open(output_pickle, "rb") as f:
        df_multires = pickle.load(f)
    processed_oids = set(df_multires['oid'].astype(str))
    print(f"Se cargaron {len(df_multires)} objetos previamente procesados.")
except (FileNotFoundError, EOFError):
    df_multires = pd.DataFrame()
    processed_oids = set()
    print("No se encontró un archivo previo. Se comenzará desde cero.")

# Lista donde se acumularán los datos
data_list = []

print("Procesando imágenes multiresolución...")

for oid in oid_subset:
    oid = str(oid)
    if oid in processed_oids:
        continue

    oid_folder = Path(multires_path) / oid
    resoluciones_multires = {}

    if oid_folder.exists() and oid_folder.is_dir():
        for fits_file in oid_folder.glob("*.fits"):
            try:
                with fits.open(fits_file) as hdul:
                    header = hdul[0].header
                    data = hdul[0].data
                    cdelt1 = header.get('CDELT1', None)
                    if data is not None and cdelt1 is not None:
                        resoluciones_multires[f"{cdelt1:.6f}"] = data
            except Exception as e:
                print(f"Error al abrir {fits_file}: {e}")

    if resoluciones_multires:
        data_list.append({'oid': oid, **resoluciones_multires})

# Crear el DataFrame final combinando con lo previo si había
if data_list:
    df_nuevos = pd.DataFrame(data_list)
    df_multires = pd.concat([df_multires, df_nuevos], ignore_index=True)

# Guardar el archivo final
with open(output_pickle, "wb") as f:
    pickle.dump(df_multires, f)

print(f"Proceso completado. Se guardaron {len(df_multires)} objetos en {output_pickle}.")
    if oid_folder.exists() and oid_folder.is_dir():
        for fits_file in oid_folder.glob("*.fits"):
            try:
                with fits.open(fits_file) as hdul:
                    header = hdul[0].header
                    data = hdul[0].data
                    cdelt1 = header.get('CDELT1', None)
                    if data is not None and cdelt1 is not None:
                        resoluciones_multires[f"{cdelt1:.6f}"] = data
            except Exception as e:
                print(f"Error al abrir {fits_file}: {e}")

    if resoluciones_multires:
        data_list.append({'oid': oid, **resoluciones_multires})

# Crear el DataFrame final combinando con lo previo si había
if data_list:
    df_nuevos = pd.DataFrame(data_list)
    df_multires = pd.concat([df_multires, df_nuevos], ignore_index=True)

# Guardar el archivo final
with open(output_pickle, "wb") as f:
    pickle.dump(df_multires, f)

print(f"Proceso completado. Se guardaron {len(df_multires)} objetos en {output_pickle}.")


Cargando el dataset de ZTF...
No se encontró un archivo previo. Se comenzará desde cero.
Procesando imágenes multiresolución...
Proceso completado. Se guardaron 32888 objetos en multires_images.pkl.


In [7]:
dataset_multires = pd.read_pickle('multires_images.pkl')

In [8]:
dataset_multires.head()

,oid,0.000069,0.000278,0.000556,0.000139,0.001111
0,ZTF18abtnlik,"[[-65.5203628540039, -96.34698486328125, -68.6...","[[-15.379009246826172, 10.055729866027832, 62....","[[116.31488037109375, 191.953369140625, 282.65...","[[-28.715036392211914, -35.20396423339844, 64....","[[-13.75033187866211, 1.7273123264312744, -3.5..."
1,ZTF20aanvpzo,"[[377.64056396484375, 409.8053894042969, 348.2...","[[122.86831665039062, 116.42054748535156, 111....","[[-2.3692266941070557, -10.608969688415527, -1...","[[424.4872131347656, 293.90380859375, 319.7689...","[[17.459169387817383, 3.769123077392578, -7.56..."
2,ZTF20aawxpru,"[[3119.797607421875, 3028.718017578125, 2880.9...","[[1058.8035888671875, 1150.329345703125, 1050....","[[-20.749204635620117, -14.779505729675293, 3....","[[4337.89892578125, 4293.310546875, 3939.08862...","[[-9.52299690246582, -30.003847122192383, -31...."
3,ZTF19abzmpvb,"[[4.907290935516357, -4.405522346496582, 76.40...","[[69.54743957519531, 81.78646087646484, 26.765...","[[92.83853149414062, 93.69679260253906, 16.684...","[[32.07027816772461, -10.95600700378418, -48.4...","[[52.04357147216797, 71.86200714111328, 149.66..."
4,ZTF20abnvdem,"[[46.263641357421875, 182.03855895996094, 63.3...","[[-15.458984375, -22.50870704650879, -11.95096...","[[-13.405747413635254, -13.405747413635254, -1...","[[-32.1228141784668, -64.35002136230469, 41.59...","[[-2.809908866882324, 8.434033393859863, 8.434..."


In [10]:
dataset_multires_2 = dataset_multires.drop_duplicates(subset = 'oid').set_index('oid', drop = True)

In [13]:
dataset_multires_2.head()
dataset_multires_2.shape
df_ZTF_2.shape

(49935, 7)

In [14]:
dataset_multires_2.to_pickle("dataset_multires.pkl")

In [16]:
df_ZTF_2['class'].value_counts()

class
bogus        15970
vs           10485
agn           9971
asteroid      9275
sn            3654
satellite      580
Name: count, dtype: int64

In [17]:
df_merged = pd.merge(df_ZTF_2, dataset_multires_2, on= 'oid', how='inner')
df_merged.shape

(32888, 12)

In [18]:
df_merged['class'].value_counts()

class
bogus       13425
agn          6449
asteroid     5128
vs           5057
sn           2829
Name: count, dtype: int64

In [19]:
df_merged.to_pickle('dataset_bimodal.pkl')